In [1]:
import tensorflow as tf

Import MLflow and enable autologging functionality.

In [2]:
import mlflow
mlflow.tensorflow.autolog()

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.28880095,  0.5619816 ,  0.16484092,  0.25850365, -0.495606  ,
        -0.47091424, -0.33911285,  0.13557343, -0.32287654,  0.28755653]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.12461472, 0.16376089, 0.11008652, 0.12089584, 0.05687295,
        0.05829472, 0.06650738, 0.10691126, 0.06759603, 0.12445974]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.8422437

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [10]:
model.fit(x_train, y_train, epochs=5)

2021/05/27 10:29:05 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'f2ebd6f8724d401d86465dc775f34a3a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.4789 - accuracy: 0.8598
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1487 - accuracy: 0.9564
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1100 - accuracy: 0.9660
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0861 - accuracy: 0.9738
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0711 - accuracy: 0.9784


Using TensorFlow backend.


INFO:tensorflow:Assets written to: /tmp/tmpxb05_m99/model/data/model/assets


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0747 - accuracy: 0.9781


[0.07466184347867966, 0.9781000018119812]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[5.37259382e-07, 1.33546250e-08, 5.42855778e-05, 6.83628372e-04,
        1.19603320e-11, 7.83288328e-07, 3.84546561e-12, 9.99252498e-01,
        7.09608628e-07, 7.56058762e-06],
       [1.86362911e-10, 2.65572511e-04, 9.99728143e-01, 6.28827456e-06,
        1.76400083e-14, 3.39142190e-08, 2.55600185e-09, 1.14760727e-11,
        1.44767318e-08, 3.10908010e-13],
       [7.06220362e-08, 9.98901725e-01, 1.38136355e-04, 7.61820684e-06,
        3.23572349e-05, 1.17531508e-05, 6.23690994e-06, 8.74598278e-04,
        2.70917481e-05, 3.44070656e-07],
       [9.99879122e-01, 1.25750577e-09, 6.07256788e-06, 1.62412803e-08,
        5.35495026e-08, 3.06554102e-06, 7.90861523e-05, 2.11728184e-05,
        5.72633496e-07, 1.10022938e-05],
       [2.72908824e-06, 9.91420812e-09, 1.72660784e-05, 2.24113261e-08,
        9.95572925e-01, 5.44574277e-06, 8.09869732e-07, 4.67727194e-04,
        1.05301915e-05, 3.92256537e-03]], dtype=float32)>